In [1]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"
database = os.environ.get('NEO4J_DATABASE')
graph = Neo4jGraph(database=database)

In [2]:
from langchain_openai import AzureOpenAIEmbeddings

embedding = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    azure_endpoint='https://sales-chatbot-llm.openai.azure.com/openai/deployments/embedding-ada-002/embeddings?api-version=2023-05-15',
    azure_deployment='text-embedding-ada-002',
    openai_api_version='2023-05-15'
)

In [ ]:
from langchain_community.vectorstores import Neo4jVector
# ! pip3 install -U langchain-huggingface
import os
os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/storage/models/embedding_models'
from langchain_huggingface import HuggingFaceEmbeddings
# Choose from https://huggingface.co/spaces/mteb/leaderboard

# embedding = HuggingFaceEmbeddings(model_name="lier007/xiaobu-embedding-v2")

model_path = os.path.join(os.environ['SENTENCE_TRANSFORMERS_HOME'], 'models--lier007--xiaobu-embedding-v2/snapshots/ee0b4ecdf5eb449e8240f2e3de2e10eeae877691')
embedding = HuggingFaceEmbeddings(model_name=model_path)

In [3]:
from langchain_community.vectorstores import Neo4jVector

lc_retrieval_query = """
WITH collect(node) as nodes
// Entity - Text Unit Mapping
WITH
collect {
    UNWIND nodes as n
    MATCH (n)<-[:HAS_ENTITY]->(c:__Chunk__)
    WITH c, count(distinct n) as freq
    RETURN c.content AS chunkText
    ORDER BY freq DESC
    LIMIT $topChunks
} AS text_mapping,
// Entity - Report Mapping
collect {
    UNWIND nodes as n
    MATCH (n)-[:IN_COMMUNITY]->(c:__Community__)
    WHERE c.summary is not null
    WITH c, c.rank as rank, c.weight AS weight
    RETURN c.summary 
    ORDER BY rank, weight DESC
    LIMIT $topCommunities
} AS report_mapping,
// Outside Relationships 
collect {
    UNWIND nodes as n
    MATCH (n)-[r]-(m) 
    WHERE NOT m IN nodes and r.description is not null
    RETURN r.description AS descriptionText
    ORDER BY r.rank, r.weight DESC 
    LIMIT $topOutsideRels
} as outsideRels,
// Inside Relationships 
collect {
    UNWIND nodes as n
    MATCH (n)-[r]-(m) 
    WHERE m IN nodes and r.description is not null
    RETURN r.description AS descriptionText
    ORDER BY r.rank, r.weight DESC 
    LIMIT $topInsideRels
} as insideRels,
// Entities description
collect {
    UNWIND nodes as n
    match (n)
    WHERE n.description is not null
    RETURN n.description AS descriptionText
} as entities
// We don't have covariates or claims here
RETURN {Chunks: text_mapping, Reports: report_mapping, 
       Relationships: outsideRels + insideRels, 
       Entities: entities} AS text, 1.0 AS score, {} AS metadata
"""

vectorstore = Neo4jVector.from_existing_graph(embedding=embedding, 
                                    index_name="embedding",
                                    node_label='__Entity__', 
                                    embedding_node_property='   embedding', 
                                    text_node_properties=['id', 'description'],
                                    retrieval_query=lc_retrieval_query)
topChunks = 3
topCommunities = 3
topOutsideRels = 10
topInsideRels = 10
topEntities = 10
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

local_search_retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.9,
                   'k': topEntities,
                   'params': {
                        "topChunks": topChunks,
                        "topCommunities": topCommunities,
                        "topOutsideRels": topOutsideRels,
                        "topInsideRels": topInsideRels,
                    }},
    tags=['GraphRAG']
)
local_search_retriever

VectorStoreRetriever(tags=['GraphRAG'], vectorstore=<langchain_community.vectorstores.neo4j_vector.Neo4jVector object at 0x14527d9a0>, search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.9, 'k': 10, 'params': {'topChunks': 3, 'topCommunities': 3, 'topOutsideRels': 10, 'topInsideRels': 10}})

In [4]:
# print(local_search_retriever.invoke('保險費暨保險單借款利息自動轉帳付款授權')[0].page_content)

Entities:
- 需填寫，繳款人為非保單關係人時須檢附關係證明文件。
- 申請金融機構轉帳繳交
- 不得為空白授權，應依是否取得保單號碼填寫相關欄位，未填寫完整須重新檢附授權書。相關規定請參照自動轉帳及信用卡付款授權書作業規範（加強身分驗證機制）。
- 首期保險費採金融機構轉帳作業應注意事項
- 經核印成功且同意承保始得進行
- 保戶約定於本公司簽約之金融機構所開立之活期帳戶扣款
- 請至各銀行臨櫃留存印鑑、核印成功便能授權扣款，土地銀行、合作金庫、第一銀行無法授權扣款, 由保戶約定於本公司簽約之金融機構所開立
- 依「新契約首期保險費繳費管道異動之繳費日認定原則」辦理
- 於核保中主動異動繳費管道者，其生效日認定依「新契約首期保險費繳費管道異動之繳費日認定原則」辦理
Reports:
Chunks:
- ，經核印成功 (且 同意承保 )始得進行首期保險費 轉帳扣款作業 。核印、扣款時間 請參照各金融機構核印、 扣款時間及相關規定 。  5. 數位帳戶 請至各銀行臨櫃留存印鑑、核印成功便能授權扣款 (土地銀行、合作金庫、第一 銀行此三家銀行無法授權扣款 )。  6. 生效日認定：   (1) 保單生效日 認定依「生效日期之認定原則 」辦理。  (2) 因核印失敗、請款失敗或 新契約於核保中主動異動繳 費管道者，其生效日認定 依「新 契約首期保險費繳費管道異動之 繳費日認定原則 」辦理。   7. 保費折扣： 以金融機構轉帳 繳交保險費
- 回目錄   6  版次： 113年07月版                                                                                            修訂日期： 113.0 7.01 參、  首期保險費繳交方式及相關規定   一、  首期保險費 -採金融機構 轉帳作業  (一) 首期保險費採 金融機構轉帳作業 應注意事項   1. 首期保險費由保戶 約定於本公司 簽約之金融機構所開立之活期帳戶扣款
- 。  2. 申請金融機構轉帳繳交 首期保險費
Relationships:
- 要保人須於授權書要保人欄位簽章並與要保書簽章相同。
- 授權人須留存金融機構之印鑑或簽章樣式。



In [4]:
vectorstore = Neo4jVector.from_existing_graph(embedding=embedding, 
                                    index_name="chunk_index",
                                    node_label='__Chunk__', 
                                    embedding_node_property='embedding', 
                                    text_node_properties=['content'])
vector_retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.9},
    tags=['RAG']
)

In [5]:
import os
# from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)
llm

AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x146eedc40>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1465a1fd0>, root_client=<openai.lib.azure.AzureOpenAI object at 0x146eee0c0>, root_async_client=<openai.lib.azure.AsyncAzureOpenAI object at 0x1465a28d0>, model_kwargs={}, openai_api_key=SecretStr('**********'), azure_endpoint='https://sales-chatbot-llm.openai.azure.com/', deployment_name='GPT4o', openai_api_version='2023-03-15-preview', openai_api_type='azure')

In [68]:
from langchain_ollama import ChatOllama
llm = ChatOllama(
    # model="llama3.1:70b-instruct-q8_0",
    model='qwen2:72b-instruct-q8_0',
)
llm

In [6]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template(
"""
你是一個有用的助手, 你的任務是整理提供的資訊, 使用長度與格式符合「multiple paragraphs」來回應使用者的問題",
提供的資訊包含 檢索內容、圖譜資料庫相關節點與關係資訊 
你必須使用繁體中文回應問題, 盡可能在500字內回應,
如果提供的資訊沒有答案或是皆無關聯, 請直接說「找不到相關資訊」並結束, 不要捏造任何資訊",
最終的回應將清理後的訊息合併成一個全面的答案，針對回應的長度和格式對所有關鍵點和含義進行解釋
根據回應的長度和格式適當添加段落和評論。以Markdown格式撰寫回應。
回應應保留原有的意思和使用的情態動詞，例如「應該」、「可以」或「將」。
請確保使用繁體中文回答問題

以下為檢索內容:
"{context}"

以下為圖譜資料庫相關節點(Entities)、關係(Relationships)、社群(Reports)、Chunks(內文節錄)資訊:
"{graph_result}"

問題: {question}
"""
)

rag_chain = (
    {"context": vector_retriever, "question": RunnablePassthrough(), "graph_result": local_search_retriever}
    | prompt
    | llm
    | StrOutputParser()
)

In [8]:
import sys
sys.path.append('..')
from tools.TokenCounter import num_tokens_from_string

q = '你對個人保險首續期繳費了解多少?'
print(num_tokens_from_string(q))

23


In [9]:
for r in rag_chain.stream(q):
    print(r, end="", flush=True)

根據提供的資訊，我可以整理出關於個人保險首續期繳費的一些關鍵點和規定。以下是詳細說明：

首先，保險公司提供多種繳費方式，其中包括以信用卡繳交保險費。如果選擇使用信用卡繳費，則需要注意以下幾點：

1. **信用卡類型**：凡持有VISA、MASTER、JCB、AE卡均可申請信用卡繳交保險費。
2. **保費折扣**：以信用卡繳交保險費的保費折扣適用規定依各險種投保規則辦理。
3. **保單限制**：一張保單號碼僅限使用一張信用卡繳費。
4. **授權書**：申請以信用卡繳交首期保險費時，需檢附「保險費信用卡付款授權書」。如果繳款人為非保單關係人，需附上關係證明文件。要保人需在授權書要保人欄位簽章，並與要保書簽章相同，授權人應留存信用卡上的簽名樣式。

其次，關於電話行銷業務透過信用卡扣繳保險費的控管機制，有以下規定：

1. **要保人與授權人同一人時**：
    - 電銷人員需在招攬過程中進行身份確認，包括電話錄音留存信用卡卡號、授權人姓名、身分證統一編號、信用卡有效期限等資訊。
    - 以複誦信用卡卡號及有效期限供授權人確認。

2. **要保人與授權人不同一人時**：
    - 電話行銷人員需取得授權人簽名的紙本或影本「保險費信用卡付款授權書」。

最後，強化保險業透過信用卡或金融機構轉帳扣款收取保險費的身份驗證機制包括：

- 保險公司及保單服務人員（含業務員、授權書建檔人員、核保人員等）需遵守相關身份驗證機制。
- 指派非銷售人員以電話回訪方式向授權人確認其姓名、身分證統一編號、信用卡卡號、有效期限等資訊是否正確，並向銀行驗證這些資訊。

總結來說，個人保險首續期繳費有明確的規定和流程，特別是使用信用卡繳費時，需注意授權書的準備和身份驗證的步驟。

如果需要進一步了解或有特定問題，歡迎隨時提出。

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


contextualize_q_system_prompt = (
    "Given a chat history and the latest user question " # 給定一段聊天歷史和使用者的最新問題
    "which might reference context in the chat history, " # 這個問題可能會引用聊天歷史中的上下文
    "formulate a standalone question which can be understood " # 請將問題重新表述為一個獨立的問題，使其在沒有聊天歷史的情況下也能被理解
    "without the chat history. Do NOT answer the question, " # 不要回答這個問題
    "just reformulate it if needed and otherwise return it as is." # 只需在必要時重新表述問題，否則原樣返回
    "請確保使用繁體中文回應"
)

# 定義上下文解析的Chain
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{question}"),
    ]
)

contextualize_chain = (
    contextualize_q_prompt
    | llm
    | StrOutputParser().with_config({
        'tags': ['contextualize_question']
    })
)


from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableParallel, RunnableLambda

store = {}

    
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 使用 RunnableParallel 來組織多個並行查詢
context_and_search_chain = RunnableParallel(
    {
        "context": RunnableLambda(lambda inputs: vector_retriever.invoke(inputs)),
        "graph_result": RunnableLambda(lambda inputs: local_search_retriever.invoke(inputs)),
        "question": lambda x: x,  # 保留原始輸入
    }
)

rag_chain = (
    contextualize_chain
    | context_and_search_chain
    | prompt
    | llm
    | StrOutputParser().with_config({
        "tags": ['final_output']
    })
)
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="chat_history"
)


In [75]:

# for r in conversational_rag_chain.stream(
#     {"input": "常見的繳費方式為何有這三種?"},
#     config={
#         "configurable": {"session_id": "abc123"}
#     },  # constructs a key "abc123" in `store`.
# ):
#     print(r , end="")

常見的保險費繳交方式包括以下幾種：

首先，首期保險費通常採用金融機構轉帳作業。保戶需要在本公司簽約的金融機構開立活期帳戶，並授權從該帳戶扣款。這種方式要求保戶在申請時提交相關授權書，授權書必須填寫完整，不得為空白授權，並且經過核印成功後，才能進行首期保險費的轉帳扣款作業。核印、扣款的時間需要參照各金融機構的相關規定。

其次，數位帳戶也是一種可行的繳費方式，但需注意的是，保戶必須親自到各銀行臨櫃留存印鑑並成功核印，才能授權扣款。目前有三家銀行（合作金庫、土地銀行、第一銀行）無法授權扣款，這意味著即使理論上可以使用數位帳戶進行轉帳，但實際操作上，還是需要到銀行臨櫃完成相關手續。

再次，保費折扣的適用規定會依各險種的投保規則辦理。如果保戶選擇金融機構轉帳繳交保險費，可能會享有一定的保費折扣。

最後，保單的生效日認定會依據「生效日期之認定原則」來辦理。如果因為核印失敗、請款失敗或新契約在核保過程中主動異動繳費管道，其生效日將依「新契約首期保險費繳費管道異動之繳費日認定原則」來辦理。

總結來說，常見的繳費方式主要包括金融機構轉帳和數位帳戶扣款。保戶需注意各金融機構的核印和扣款時間，並依照保單規定的程序辦理，以確保繳費順利完成。

In [12]:
from langchain_core.output_parsers.string import StrOutputParser
from fastapi import FastAPI
from langserve import add_routes
from typing import List, Tuple
from pydantic import BaseModel, Field

class ChatHistory(BaseModel):
    """Chat history with the bot."""
    question: str
    
conversational_rag_chain = (
  conversational_rag_chain | StrOutputParser()
).with_types(input_type=ChatHistory)

# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 5. Adding chain route

add_routes(
    app,
    conversational_rag_chain
)

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="localhost", port=8000)

RuntimeError: asyncio.run() cannot be called from a running event loop